# Books Recommendetion Project - 398

## In this project we using clustering for books recommendation

## Collaborative filtering

## Content

###The Book-Crossing dataset comprises 3 files.

###● Users

Contains the users. Note that user IDs (User-ID) have been anonymized and map to integers.
Demographic data is provided (Location, Age) if available. Otherwise, these fields contain NULL-
values.

###● Books

Books are identified by their respective ISBN. Invalid ISBNs have already been removed from the
dataset. Moreover, some content-based information is given (Book-Title, Book-Author, Year-
Of-Publication, Publisher), obtained from Amazon Web Services. Note that in case of several
authors, only the first is provided. URLs linking to cover images are also given, appearing in three
different flavours (Image-URL-S, Image-URL-M, Image-URL-L), i.e., small, medium, large. These
URLs point to the Amazon web site.

###● Ratings

Contains the book rating information. Ratings (Book-Rating) are either explicit, expressed on a
scale from 1-10 (higher values denoting higher appreciation), or implicit, expressed by 0.

In [2]:
# importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# load the books dataset
books = pd.read_csv("Books.csv")

<ipython-input-3-e202a6eed0a7>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("Books.csv")


In [4]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [5]:
# checking tha shape of dataframe
books.shape

(271360, 8)

In [6]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

### Conclution:

Here Image URL columns is important for the poster. So, we will keep it

In [7]:
books = books[['ISBN','Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher','Image-URL-L']]

In [8]:
# Lets rename some weird columns name
books.rename(columns={"Book-Title":"title","Book-Author":"author",
                      "Year-Of-Publication":"year",
                      "Publisher":"publisher","Image-URL-L":"image_url"},inplace= True)

<ipython-input-8-998434d9db53>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books.rename(columns={"Book-Title":"title","Book-Author":"author",


In [9]:
books.head()

,ISBN,title,author,year,publisher,image_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...


## Now load the user dataset

In [10]:
users = pd.read_csv("Users.csv")

In [11]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [12]:
# checking the shape of dataframe
users.shape

(278858, 3)

In [13]:
# lets rename the some weird columns name
users.rename(columns={"User-ID":'user-id',"Location":"lacation",
                      "Age":"age"},inplace =True)

In [14]:
users.head()

,user-id,lacation,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


#Lets load the Rating datasets

In [15]:
ratings= pd.read_csv("Ratings.csv")
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [16]:
# checking the shape of the dataframe
ratings.shape

(1149780, 3)

In [17]:
# Rename the columns name
ratings.rename(columns={"User-ID":"user-id","Book-Rating":"rating"},inplace=True)

In [18]:
ratings.head(10)

,user-id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
5,276733,2080674722,0
6,276736,3257224281,8
7,276737,0600570967,6
8,276744,038550120X,7
9,276745,342310538,10


# Now we have 3 dataframes here



*   books
*   users
*   ratings



In [19]:
# lets check again all the shape of dataframes
print(books.shape, users.shape, ratings.shape, sep="\n")

(271360, 6)
(278858, 3)
(1149780, 3)


### Now we will check value counts of given rating by users

In [20]:
ratings['user-id'].value_counts()

user-id
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: count, Length: 105283, dtype: int64

In [21]:
# checking the shape
ratings['user-id'].value_counts().shape

(105283,)

In [22]:
# checking the unique values
ratings['user-id'].unique().shape

(105283,)

In [23]:
# Lets store users who had at least rated more than 200 books
x = ratings['user-id'].value_counts() > 200

In [24]:
x[x].shape

(899,)

In [25]:
y= x[x].index

In [26]:
y

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727, 268622,
       188951],
      dtype='int64', name='user-id', length=899)

In [27]:
# lets create a new columns for rating more than 200 books
ratings = ratings[ratings['user-id'].isin(y)]

In [28]:
ratings.head()

,user-id,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0


In [29]:
ratings.shape

(526356, 3)

# Now join ratings with books

Here we remove the ISBN column because this columns is same column in both dataframes

In [30]:
ratings_with_books = ratings.merge(books, on='ISBN')

In [31]:
ratings_with_books.head()

,user-id,ISBN,rating,title,author,year,publisher,image_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...


In [32]:
ratings_with_books.shape

(487671, 8)

In [33]:
# lets make group of title and rating for better understanding
number_rating = ratings_with_books.groupby("title")['rating'].count().reset_index()

In [34]:
number_rating.head()

,title,rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [35]:
# rename the rating columns
number_rating.rename(columns={'rating':'num_of_rating'},inplace=True)

In [36]:
number_rating.head()

,title,num_of_rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


### now we merge the rating with books column and number rating columns

In [37]:
final_rating = ratings_with_books.merge(number_rating, on='title')

In [38]:
final_rating.head()

,user-id,ISBN,rating,title,author,year,publisher,image_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


In [39]:
# checking the shape of dataframe
final_rating.shape

(487671, 9)

In [40]:
# Lets take those books which got at least 60 rating of user
final_rating = final_rating[final_rating['num_of_rating'] >= 60]

In [41]:
final_rating.head()

,user-id,ISBN,rating,title,author,year,publisher,image_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


In [42]:
final_rating.shape

(50271, 9)

In [43]:
# lets drop the duplicates
final_rating.drop_duplicates(['user-id','title'],inplace=True)

In [44]:
final_rating.shape

(48571, 9)

# Lets create a pivot table

A pivot table is a data summarization tool that allows you to rearrange and summarize the data in a table format. It is often used to create reports and dashboards.

Pivot tables are created by selecting a set of data and then specifying the rows, columns, and values that you want to include in the table. You can also specify the aggregation function that you want to use to summarize the data (e.g., sum, average, count).

In [45]:
book_pivot = final_rating.pivot_table(columns='user-id', index='title', values= 'rating')

In [46]:
book_pivot

user-id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A Bend in the Road,0.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wish You Well,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Without Remorse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Wuthering Heights,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
book_pivot.shape

(527, 885)

In [48]:
# fill null values with 0
book_pivot.fillna(0, inplace=True)

In [49]:
book_pivot.head()

user-id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Here we completed the Exploratory data analysis part of this dataframe now we create models

In [59]:
book_pivot.shape

(527, 885)

# Training model

In [51]:
from scipy.sparse import csr_matrix
book_sparse = csr_matrix (book_pivot)
type(book_sparse)

scipy.sparse._csr.csr_matrix

## Now import our clustering algotithm which is nearest neighbors this is an unsupervised machine learning algorithm.

In [54]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute')

In [56]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [57]:
distance, suggestion = model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1),n_neighbors=6)

In [58]:
distance

array([[ 0.        , 39.47150871, 39.62322551, 39.8246155 , 40.81666326,
        40.91454509]])

In [60]:
suggestion

array([[237,   7, 441, 133, 277, 487]])

In [62]:
book_pivot.iloc[251,:]

user-id
254       0.0
2276      0.0
2766      0.0
2977      0.0
3363      0.0
         ... 
275970    0.0
277427    0.0
277478    0.0
277639    0.0
278418    0.0
Name: N Is for Noose, Length: 885, dtype: float64

In [63]:
for i in range (len(suggestion)):
  print(book_pivot.index[suggestion[i]])

Index(['Midnight in the Garden of Good and Evil', 'A Civil Action', 'The Reef',
       'Fat Tuesday', 'Pleading Guilty',
       'Tom Clancy's Op-Center (Tom Clancy's Op Center (Paperback))'],
      dtype='object', name='title')


In [64]:
book_pivot.index[3]

'4 Blondes'

In [65]:
# keeping books name
book_names = book_pivot.index

In [67]:
book_names[2]

'2nd Chance'

In [69]:
np.where(book_pivot.index == '4 Blondes')[0][0]

3

# Find Url

In [72]:
# final_rating['title'].value_counts()
ids  = np.where(final_rating['title'] == 'Harry Potter and the Chamber of Secrets (Book 2)')[0][0]

In [73]:
final_rating.iloc[ids]['image_url']

'http://images.amazon.com/images/P/0439064872.01.LZZZZZZZ.jpg'

In [74]:
book_name = []
for book_id in suggestion:
  book_name.append(book_pivot.index[book_id])

In [75]:
book_name[0]

Index(['Midnight in the Garden of Good and Evil', 'A Civil Action', 'The Reef',
       'Fat Tuesday', 'Pleading Guilty',
       'Tom Clancy's Op-Center (Tom Clancy's Op Center (Paperback))'],
      dtype='object', name='title')

In [78]:
ids_index = []
for name in book_name[0]:
  ids = np.where (final_rating['title'] == name)[0][0]
  ids_index.append(ids)

In [80]:
for idx in ids_index :
  url = final_rating.iloc[idx]['image_url']
  print(url)

http://images.amazon.com/images/P/0679751521.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0679772677.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/051512608X.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0553478257.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0446365505.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0425147363.01.LZZZZZZZ.jpg


In [82]:
import pickle
import os # Import the os module for directory creation

# Create the artifacts directory if it doesn't exist
os.makedirs('artifacts', exist_ok=True)

# Now you can safely write to the files
pickle.dump(model,open('artifacts/model.pkl','wb'))
pickle.dump(book_names,open('artifacts/book_names.pkl','wb'))
pickle.dump(final_rating,open('artifacts/final_rating.pkl','wb'))
pickle.dump(book_pivot,open('artifacts/book_pivot.pkl','wb'))

# Testing model

In [85]:
def recommend_book(book_name) :
  book_id =np.where(book_pivot.index == book_name)[0][0]
  distance, suggestion = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1), n_neighbors = 6)

  for i in range (len(suggestion)):
    books = book_pivot.index[suggestion[i]]
    for j in books :
      if j == book_name:
        print(f"you searched '{book_name}'\n")
        print('The suggestion books are : \n')
      else:
        print(j)

In [86]:
book_name = 'Harry Potter and the Chamber of Secrets (Book 2)'
recommend_book(book_name)

you searched 'Harry Potter and the Chamber of Secrets (Book 2)'

The suggestion books are : 

Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Sorcerer's Stone (Book 1)
Tom Clancy's Op-Center (Tom Clancy's Op Center (Paperback))
Truly, Madly Manhattan


In [87]:
book_name = 'A Civil Action'
recommend_book(book_name)

you searched 'A Civil Action'

The suggestion books are : 

Pleading Guilty
Journey
Last Man Standing
The Most Wanted
Primary Colors: A Novel of Politics


## Conlusion :

### Our book recommendation system project successfully integrates collaborative filtering and clustering models to enhance the accuracy and relevance of book suggestions for users. By leveraging collaborative filtering, we harness the power of user interactions and preferences to identify patterns and relationships between different books. This method ensures that recommendations are personalized, aligning closely with the tastes and interests of individual users.

### The addition of a clustering model further refines our recommendation system by grouping similar users or books into clusters. This approach allows us to provide recommendations that are not only personalized but also diverse, by exposing users to books from different clusters that they might not have otherwise discovered. The clustering model enhances the system's ability to cater to niche interests and broadens the spectrum of recommendations.